### **TÍTULO**

Import libraries

In [1]:
import os
import numpy as np
from tensorflow import keras

# Import data

In [4]:
train_path = r"..\Data\BoneFractureYolo8\train"
test_path = r"..\Data\BoneFractureYolo8\test"
valid_path = r"..\Data\BoneFractureYolo8\valid"

Importar imágenes
Procesar imágenes procesadas
Importar el yaml con la simágenes porcesadas
Entrenar modleo preentrenado